# Extractions for observed forest disturbances

In [ ]:
# import required modules
import numpy as np
import geopandas as gpd
import rasterio as rst
from rasterstats import zonal_stats
import glob
import os
import pandas as pd
import re
import multiprocessing
from multiprocessing import Pool

## EFFIS data

The id column is cat in case earlier versions of the extractions need to be traced back.
There was a problem with the year 2017. The folder amended files contains an amended version of the dataset.
Original files provied by guido are stored in the external hard drive marcodata2019 under ESSDrive/datasets/guido

In [ ]:
# read in polygon data
results_final = []
for filep in glob.glob('/mnt/data1tb/Dropbox/Forest@risk/polygonsGRASS/EFFIS20205year_amended/*.shp'):
    # read in polygon
    tmpp = gpd.read_file(filep)
    # append polygon to list
    results_final.append(tmpp)

### Static datasets

#### PFTs

In [ ]:
# Loop through raster files
for filer in glob.glob('/mnt/data1tb/rasters/EU/pfts/*.tif'):
        # get out names for variable of interest
        varname = os.path.basename(filer).split('.')[0]
        # print variable name
        print(varname)
        # open raster with rasterio
        tmpr = rst.open(filer)
        # convert into array
        tmpar = tmpr.read(1)
        # Loop through vector files
        for filep in results_final:
            # zonal statisticfileslegs (sum)
            stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['sum'], all_touched=True,nodata=tmpr.nodata)
            # zonal statistics (count)
            totpixels = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['count'], all_touched=True,nodata=tmpr.nodata)
            # convert dictionaries into lists
            stats1 = [val for dic in stats for val in dic.values()]
            totpixels1 = [val for dic in totpixels for val in dic.values()]
            # store in pandas dataframe
            filep[varname] = stats1
            # total number of pixels
            varname1 = varname + '_pixels'
            filep[varname1] = totpixels1

#### Continuous variables

In [ ]:
# Loop through raster files
for filer in glob.glob('/mnt/data1tb/rasters/EU/static/*.tif'):
    # get out names for variable of interest
    varname = os.path.basename(filer).split('.')[0]
    # print variable name
    print(varname)
    # open raster with rasterio
    tmpr = rst.open(filer)
    # convert into array
    tmpar = tmpr.read(1)
    # Loop through vector files
    for filep in results_final:
        # zonal statistics (mean)
        stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['mean'], all_touched=True, nodata=tmpr.nodata)
        # convert dictionaries into lists
        stats1 = [val for dic in stats for val in dic.values()]
        # store in pandas dataframe
        filep[varname] = stats1

#### EFI tree cover maps

In [ ]:
# Loop through raster files
for filer in glob.glob('/mnt/data1tb/rasters/EU/EFItrees/*.tif'):
        # open raster with rasterio
        tmpr = rst.open(filer)
        # convert into array
        tmpar = tmpr.read(1)
        # get out names for variable of interest
        varname = os.path.basename(filer).split('.')[0]
        print(varname)
        # Loop through vector files
        for filep in results_final:
            # zonal statistics (mean)
            stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['sum'], all_touched=True, nodata=tmpr.nodata)
            # convert dictionaries into lists
            stats1 = [val for dic in stats for val in dic.values()]
            # store in pandas dataframe
            filep[varname] = stats1

###  Spatio-temporal datasets 

#### Event-scale variables

In [ ]:
# Loop through raster files
for filer in glob.glob('/mnt/data1tb/rasters/EU/dynamic1_Nov19/*.tif'):
    # open raster with rasterio
    tmpr = rst.open(filer)
    # convert into array
    tmpar = tmpr.read(1)
    # if integer convert into floating point
    if ('float' in tmpar.dtype.type.__name__)==False:
        # set array values to floating
        tmpar = tmpar.astype('float')
    # exception for Aridity Index (lack consistent naming)
    if 'AI' in os.path.basename(filer):
        varname = os.path.basename(filer).split('.')[0].split('_')[0]
        year = os.path.basename(filer).split('.')[0].split('_')[1]
    # contains population name
    elif 'pop' in os.path.basename(filer):
           varname = os.path.basename(filer).split('.')[0].split('2')[0]
           numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
           # extract the number with the longest number of digits
           year = max(numbs, key=len)
    elif 'suppressionp' in os.path.basename(filer) or 'ignitionp' in os.path.basename(filer):
               varname = os.path.basename(filer).split('.')[0].split('_')[0]
               numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
               # extract the number with the longest number of digits
               year = max(numbs, key=len)
    else:
        # get out names for variable of interest
        varnametmp = os.path.basename(filer).split('.')[0]
        varname = varnametmp.split('_')[0] + "_" + varnametmp.split('_')[1] + "_" + varnametmp.split('_')[3]
        # get out name for variable of interest
        # parse numbers (could be year, resolution or season!)
        numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
        # extract the number with the longest number of digits
        year = max(numbs, key=len)
    # match year with polygon
    for filep in results_final:
        # create year filter
        if (str(int(filep.yearssn.unique()[0])) == year):
            # zonal statistics (mean)
            stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['mean'], all_touched=True,nodata=tmpr.nodata)
            # convert dictionaries into lists
            stats1 = [val for dic in stats for val in dic.values()]
            # store in pandas dataframe
            filep[varname] = stats1
        else:
            pass

#### Legacy effects: includes legacy effects back to 5 years before the year of interest

In [ ]:
# exclude population density and fire suppression and ignition probabilities
filesleg = [fn for fn in glob.glob('/mnt/data1tb/rasters/EU/dynamic1_Nov19/*.tif') if
 'pop' not in os.path.basename(fn) and 'suppressionp' not in os.path.basename(fn) and 'ignitionp' not in os.path.basename(fn)]

# Loop through raster files
for filer in filesleg:
    # exception for Aridity Index (lack consistent naming)
    if 'AI' in os.path.basename(filer):
        varname = os.path.basename(filer).split('.')[0].split('_')[0]
        year = os.path.basename(filer).split('.')[0].split('_')[1]
    else:
        # get out names for variable of interest
        varnametmp = os.path.basename(filer).split('.')[0]
        varname = varnametmp.split('_')[0] + "_" + varnametmp.split('_')[1] + "_" + varnametmp.split('_')[3]
        # parse numbers (could be year, resolution or season!)
        numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
        # extract the number with the longest number of digits
        year = int(max(numbs, key=len))
        if year <= 1999:
           pass
        else:
            # --- create list of lagged rasters --- (goes back 5 years)
            rastlags = {} 
            lags = list(range(1,5+1))
            for lag in lags:
                # strings for lagged effects and year of interest
                year_m1 = str(year-lag)
                # path for raster year
                fpathym1 = '/mnt/data1tb/rasters/EU/dynamic1_Nov19/' + str.replace(varnametmp, str(year),year_m1)+'.tif'
                # open raster with rasterio
                tmpr = rst.open(fpathym1)
                # convert into array
                tmpar = tmpr.read(1)
                # if integer convert into floating point
                if ('float' in tmpar.dtype.type.__name__) == False:             
                    # set array values to floating
                    tmpar = tmpar.astype('float')
                # append raster to list
                rastlags[year_m1] = tmpar
            # --- go through each set of polygons divided by year
            for filep in results_final:
                # create year filter
                if (filep.yearssn.unique()[0] == year):
                    # --- zonal statistics for lagged effects
                    for onerast in rastlags:
                        # zonal statistics (mean)
                        stats = zonal_stats(filep, rastlags[onerast], affine=tmpr.transform, stats=['mean'], all_touched=True,nodata=tmpr.nodata)
                        # convert dictionaries into lists
                        stats1 = [val for dic in stats for val in dic.values()]
                        # convert dictionaries into lists
                        filep[varname+'_ym'+str(year - int(onerast))] = stats1

#### Write out file

In [ ]:
# concatenate list of pandas dataframes
results_final1 = pd.concat(results_final)
# delete geometry column
results_final1 = results_final1.drop(['geometry'], axis=1)
# write results as csv file
results_final1.to_csv('/mnt/data1tb/Dropbox/Forest@risk/scripts/y2020/R/outdata/observed/EFFIS_OBSERVED.csv', index=False)

## WINDFOR data

The id column is Id_poly in case earlier versions of the extractions need to be traced back.

In [ ]:
# read in polygon data
results_final = []
for filep in glob.glob('/mnt/data1tb/rasters/guido/WIND/*.shp'):
    # read in polygon
    tmpp = gpd.read_file(filep)
    # append polygon to list
    results_final.append(tmpp)

### Static datasets

#### PFTs

In [ ]:
# Loop through raster files
for filer in glob.glob('/mnt/data1tb/rasters/EU/pfts/*.tif'):
        # get out names for variable of interest
        varname = os.path.basename(filer).split('.')[0]
        # print variable name
        print(varname)
        # open raster with rasterio
        tmpr = rst.open(filer)
        # convert into array
        tmpar = tmpr.read(1)
        # Loop through vector files
        for filep in results_final:
            # zonal statisticfileslegs (sum)
            stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['sum'], all_touched=True,nodata=tmpr.nodata)
            # zonal statistics (count)
            totpixels = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['count'], all_touched=True,nodata=tmpr.nodata)
            # convert dictionaries into lists
            stats1 = [val for dic in stats for val in dic.values()]
            totpixels1 = [val for dic in totpixels for val in dic.values()]
            # store in pandas dataframe
            filep[varname] = stats1
            # total number of pixels
            varname1 = varname + '_pixels'
            filep[varname1] = totpixels1

#### Continuous variables

In [ ]:
# Loop through raster files
for filer in glob.glob('/mnt/data1tb/rasters/EU/static/*.tif'):
    # get out names for variable of interest
    varname = os.path.basename(filer).split('.')[0]
    # print variable name
    print(varname)
    # open raster with rasterio
    tmpr = rst.open(filer)
    # convert into array
    tmpar = tmpr.read(1)
    # Loop through vector files
    for filep in results_final:
        # zonal statistics (mean)
        stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['mean'], all_touched=True, nodata=tmpr.nodata)
        # convert dictionaries into lists
        stats1 = [val for dic in stats for val in dic.values()]
        # store in pandas dataframe
        filep[varname] = stats1

#### EFI tree cover maps

In [ ]:
# Loop through raster files
for filer in glob.glob('/mnt/data1tb/rasters/EU/EFItrees/*.tif'):
        # open raster with rasterio
        tmpr = rst.open(filer)
        # convert into array
        tmpar = tmpr.read(1)
        # get out names for variable of interest
        varname = os.path.basename(filer).split('.')[0]
        print(varname)
        # Loop through vector files
        for filep in results_final:
            # zonal statistics (mean)
            stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['sum'], all_touched=True, nodata=tmpr.nodata)
            # convert dictionaries into lists
            stats1 = [val for dic in stats for val in dic.values()]
            # store in pandas dataframe
            filep[varname] = stats1

###  Spatio-temporal datasets 

#### Event-scale variables

In [ ]:
# Loop through raster files
for filer in glob.glob('/mnt/data1tb/rasters/EU/dynamic1_Nov19/*.tif'):
    # open raster with rasterio
    tmpr = rst.open(filer)
    # convert into array
    tmpar = tmpr.read(1)
    # if integer convert into floating point
    if ('float' in tmpar.dtype.type.__name__)==False:
        # set array values to floating
        tmpar = tmpar.astype('float')
    # exception for Aridity Index (lack consistent naming)
    if 'AI' in os.path.basename(filer):
        varname = os.path.basename(filer).split('.')[0].split('_')[0]
        year = os.path.basename(filer).split('.')[0].split('_')[1]
    # contains population name
    elif 'pop' in os.path.basename(filer):
           varname = os.path.basename(filer).split('.')[0].split('2')[0]
           numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
           # extract the number with the longest number of digits
           year = max(numbs, key=len)
    elif 'suppressionp' in os.path.basename(filer) or 'ignitionp' in os.path.basename(filer):
               varname = os.path.basename(filer).split('.')[0].split('_')[0]
               numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
               # extract the number with the longest number of digits
               year = max(numbs, key=len)
    else:
        # get out names for variable of interest
        varnametmp = os.path.basename(filer).split('.')[0]
        varname = varnametmp.split('_')[0] + "_" + varnametmp.split('_')[1] + "_" + varnametmp.split('_')[3]
        # get out name for variable of interest
        # parse numbers (could be year, resolution or season!)
        numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
        # extract the number with the longest number of digits
        year = max(numbs, key=len)
        # match year with polygon
        for filep in results_final:
            # create year filter
            if (str(filep.yearssn.unique()[0]) == year):
                # zonal statistics (mean)
                stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['mean'], all_touched=True,nodata=tmpr.nodata)
                # convert dictionaries into lists
                stats1 = [val for dic in stats for val in dic.values()]
                # store in pandas dataframe
                filep[varname] = stats1
            else:
                pass

#### Legacy effects 

In [ ]:
# exclude population density and fire suppression and ignition probabilities
filesleg = [fn for fn in glob.glob('/mnt/data1tb/rasters/EU/dynamic1_Nov19/*.tif') if
 'pop' not in os.path.basename(fn) and 'suppressionp' not in os.path.basename(fn) and 'ignitionp' not in os.path.basename(fn)]

# Loop through raster files
for filer in filesleg:
    # exception for Aridity Index (lack consistent naming)
    if 'AI' in os.path.basename(filer):
        varname = os.path.basename(filer).split('.')[0].split('_')[0]
        year = os.path.basename(filer).split('.')[0].split('_')[1]
    else:
        # get out names for variable of interest
        varnametmp = os.path.basename(filer).split('.')[0]
        varname = varnametmp.split('_')[0] + "_" + varnametmp.split('_')[1] + "_" + varnametmp.split('_')[3]
        # parse numbers (could be year, resolution or season!)
        numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
        # extract the number with the longest number of digits
        year = int(max(numbs, key=len))
        if year<= 1999:
           pass
        else:
            #print(varnametmp)
            # extract year-1 from name
            year_m1 = str(year-1)
            year = str(year)
            # path for raster year -1
            fpathym1 = '/mnt/data1tb/rasters/EU/dynamic1_Nov19/' + str.replace(varnametmp, year, year_m1) + '.tif'
            # open raster with rasterio
            tmpr = rst.open(fpathym1)
            # convert into array
            tmpar = tmpr.read(1)
            # if integer convert into floating point
            if ('float' in tmpar.dtype.type.__name__) == False:
               # set array values to floating
               tmpar = tmpar.astype('float')
            # match year with polygon
            for filep in results_final:
                # create year filter
                if (str(filep.yearssn.unique()[0]) == year):
                    # zonal statistics (mean)
                    stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['mean'], all_touched=True,nodata=tmpr.nodata)
                    # convert dictionaries into lists
                    stats1 = [val for dic in stats for val in dic.values()]
                    # store in pandas dataframe
                    filep[varname+'_ym1'] = stats1

In [ ]:
# concatenate list of pandas dataframes
results_final1 = pd.concat(results_final)
# delete geometry column
results_final1 = results_final1.drop(['geometry'], axis=1)
# write results as csv file
results_final1.to_csv('/home/marco/Desktop/WIND_OBSERVED.csv', index=False)

## INSECT data

The id column is cat in case earlier versions of the extractions need to be traced back.

In [ ]:
# read in polygon data
results_final = []
for filep in glob.glob('/mnt/data1tb/Dropbox/Forest@risk/polygonsGRASS/USDA/*.shp'):
    # read in polygon
    tmpp = gpd.read_file(filep)
    # append polygon to list
    results_final.append(tmpp)

#### PFTs

In [ ]:
# Loop through raster files
for filer in glob.glob('/mnt/data1tb/rasters/EU/pfts/*.tif'):
        # get out names for variable of interest
        varname = os.path.basename(filer).split('.')[0]
        # print variable name
        print(varname)
        # open raster with rasterio
        tmpr = rst.open(filer)
        # convert into array
        tmpar = tmpr.read(1)
        # Loop through vector files
        for filep in results_final:
            # zonal statisticfileslegs (sum)
            stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['sum'], all_touched=True,nodata=tmpr.nodata)
            # zonal statistics (count)
            totpixels = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['count'], all_touched=True,nodata=tmpr.nodata)
            # convert dictionaries into lists
            stats1 = [val for dic in stats for val in dic.values()]
            totpixels1 = [val for dic in totpixels for val in dic.values()]
            # store in pandas dataframe
            filep[varname] = stats1
            # total number of pixels
            varname1 = varname + '_pixels'
            filep[varname1] = totpixels1

#### Continuous variables

In [ ]:
# Loop through raster files
for filer in glob.glob('/mnt/data1tb/rasters/NA/static/*.tif'):
    # get out names for variable of interest
    varname = os.path.basename(filer).split('.')[0]
    # print variable name
    print(varname)
    # open raster with rasterio
    tmpr = rst.open(filer)
    # convert into array
    tmpar = tmpr.read(1)
    # Loop through vector files
    for filep in results_final:
        # zonal statistics (mean)
        stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['mean'], all_touched=True, nodata=tmpr.nodata)
        # convert dictionaries into lists
        stats1 = [val for dic in stats for val in dic.values()]
        # store in pandas dataframe
        filep[varname] = stats1

###  Spatio-temporal datasets 

#### Event-scale variables

In [ ]:
# Loop through raster files
for filer in glob.glob('/mnt/data1tb/rasters/NA/dynamic/*.tif'):
    # open raster with rasterio
    tmpr = rst.open(filer)
    # convert into array
    tmpar = tmpr.read(1)
    # if integer convert into floating point
    if ('float' in tmpar.dtype.type.__name__)==False:
        # set array values to floating
        tmpar = tmpar.astype('float')
    # exception for Aridity Index (lack consistent naming)
    if 'AI' in os.path.basename(filer):
        varname = os.path.basename(filer).split('.')[0].split('_')[0]
        year = os.path.basename(filer).split('.')[0].split('_')[1]
    # contains population name
    elif 'pop' in os.path.basename(filer):
           varname = os.path.basename(filer).split('.')[0].split('2')[0]
           numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
           # extract the number with the longest number of digits
           year = max(numbs, key=len)
    elif 'suppressionp' in os.path.basename(filer) or 'ignitionp' in os.path.basename(filer):
               varname = os.path.basename(filer).split('.')[0].split('_')[0]
               numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
               # extract the number with the longest number of digits
               year = max(numbs, key=len)
    else:
        # get out names for variable of interest
        varnametmp = os.path.basename(filer).split('.')[0]
        varname = varnametmp.split('_')[0] + "_" + varnametmp.split('_')[1] + "_" + varnametmp.split('_')[3]
        # get out name for variable of interest
        # parse numbers (could be year, resolution or season!)
        numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
        # extract the number with the longest number of digits
        year = max(numbs, key=len)
        # match year with polygon
    # match year with polygon
    for filep in results_final:
        # create year filter
        if (str(int(filep.yearssn.unique()[0])) == year):
            # zonal statistics (mean)
            stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['mean'], all_touched=True,nodata=tmpr.nodata)
            # convert dictionaries into lists
            stats1 = [val for dic in stats for val in dic.values()]
            # store in pandas dataframe
            filep[varname] = stats1
        else:
            pass

#### Legacy effects 

In [ ]:
# exclude population density and fire suppression and ignition probabilities
filesleg = [fn for fn in glob.glob('/mnt/data1tb/rasters/EU/dynamic1_Nov19/*.tif') if
 'pop' not in os.path.basename(fn) and 'suppressionp' not in os.path.basename(fn) and 'ignitionp' not in os.path.basename(fn)]

# Loop through raster files
for filer in filesleg:
    # exception for Aridity Index (lack consistent naming)
    if 'AI' in os.path.basename(filer):
        varname = os.path.basename(filer).split('.')[0].split('_')[0]
        year = os.path.basename(filer).split('.')[0].split('_')[1]
    else:
        # get out names for variable of interest
        varnametmp = os.path.basename(filer).split('.')[0]
        varname = varnametmp.split('_')[0] + "_" + varnametmp.split('_')[1] + "_" + varnametmp.split('_')[3]
        # parse numbers (could be year, resolution or season!)
        numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
        # extract the number with the longest number of digits
        year = int(max(numbs, key=len))
        if year<= 1999:
           pass
        else:
            #print(varnametmp)
            # extract year-1 from name
            year_m1 = str(year-1)
            year = str(year)
            # path for raster year -1
            fpathym1 = '/mnt/data1tb/rasters/EU/dynamic1_Nov19/' + str.replace(varnametmp, year, year_m1) + '.tif'
            # open raster with rasterio
            tmpr = rst.open(fpathym1)
            # convert into array
            tmpar = tmpr.read(1)
            # if integer convert into floating point
            if ('float' in tmpar.dtype.type.__name__) == False:
               # set array values to floating
               tmpar = tmpar.astype('float')
            # match year with polygon
            for filep in results_final:
                # create year filter
                if (str(filep.yearssn.unique()[0]) == year):
                    # zonal statistics (mean)
                    stats = zonal_stats(filep, tmpar, affine=tmpr.transform, stats=['mean'], all_touched=True,nodata=tmpr.nodata)
                    # convert dictionaries into lists
                    stats1 = [val for dic in stats for val in dic.values()]
                    # store in pandas dataframe
                    filep[varname+'_ym1'] = stats1

In [ ]:
# ----Dynamic maps: legacy effects

# exclude population density and fire suppression and ignition probabilities
filesleg = [fn for fn in glob.glob('/ESS_Datasets/USERS/Marco/rasters/NA/dynamic/*.tif') if
 'pop' not in os.path.basename(fn) and 'suppressionp' not in os.path.basename(fn) and 'ignitionp' not in os.path.basename(fn)]

# Loop through raster files
for filer in filesleg:
    # open raster with rasterio
    tmpr = rst.open(filer)
    # get out names for variable of interest
    varnametmp = os.path.basename(filer).split('.')[0]
    varname = varnametmp.split('_')[0] + "_" + varnametmp.split('_')[1] + "_" + varnametmp.split('_')[3]
    # parse numbers (could be year, resolution or season!)
    numbs = re.findall('\d+', os.path.basename(filer).split('.')[0])
    # extract the number with the longest number of digits
    year = max(numbs, key=len)
    if int(year) < 2000:
        pass
    else:
        #------- Year-1
        year_m1 = str(int(year) - 1)
        # path for raster year -1
        fpathym1 = '/ESS_Datasets/USERS/Marco/rasters/NA/dynamic/' + str.replace(varnametmp, year, year_m1) + '.tif'
        # open raster with rasterio
        tmpr = rst.open(fpathym1)
        # convert into array
        tmpar = tmpr.read(1)
        # if integer convert into floating point
        # get transformation parameter (needed for zonal stats)
        affine = tmpr.transform
        # match year with polygon
        for filep in results_final:
            # create year filter
            if (str(int(filep.YEAR.unique()[0])) == year):
                # zonal statistics (mean)
                stats = zonal_stats(filep, tmpar, affine=affine, stats=['mean'], all_touched=True,  nodata=tmpr.nodata)
                # convert dictionaries into lists
                stats1 = [val for dic in stats for val in dic.values()]
                # store in pandas dataframe
                filep[varname + '_ym1'] = stats1
        # ------- Year-2
        year_m2 = str(int(year) - 2)
        # path for raster year -1
        fpathym2 = '/ESS_Datasets/USERS/Marco/rasters/NA/dynamic/' + str.replace(varnametmp, year, year_m2) + '.tif'
        # open raster with rasterio
        tmpr = rst.open(fpathym2)
        # convert into array
        tmpar = tmpr.read(1)
        # if integer convert into floating point
        # get transformation parameter (needed for zonal stats)
        affine = tmpr.transform
        # match year with polygon
        for filep in results_final:
            # create year filter
            if (str(int(filep.YEAR.unique()[0])) == year):
                # zonal statistics (mean)
                stats = zonal_stats(filep, tmpar, affine=affine, stats=['mean'], all_touched=True, nodata=tmpr.nodata)
                # convert dictionaries into lists
                stats1 = [val for dic in stats for val in dic.values()]
                # store in pandas dataframe
                filep[varname + '_ym2'] = stats1
        # ------- Year-3
        year_m3 = str(int(year) - 3)
        # path for raster year -1
        fpathym3 = '/ESS_Datasets/USERS/Marco/rasters/NA/dynamic/' + str.replace(varnametmp, year, year_m3) + '.tif'
        # open raster with rasterio
        tmpr = rst.open(fpathym3)
        # convert into array
        tmpar = tmpr.read(1)
        # if integer convert into floating point
        # get transformation parameter (needed for zonal stats)
        affine = tmpr.transform
        # match year with polygon
        for filep in results_final:
            # create year filter
            if (str(int(filep.YEAR.unique()[0])) == year):
                # zonal statistics (mean)
                stats = zonal_stats(filep, tmpar, affine=affine, stats=['mean'], all_touched=True, nodata=tmpr.nodata)
                # convert dictionaries into lists
                stats1 = [val for dic in stats for val in dic.values()]
                # store in pandas dataframe
                filep[varname + '_ym3'] = stats1
        # ------- Year-4
        year_m4 = str(int(year) - 4)
        # path for raster year -1
        fpathym4 = '/ESS_Datasets/USERS/Marco/rasters/NA/dynamic/' + str.replace(varnametmp, year, year_m4) + '.tif'
        # open raster with rasterio
        tmpr = rst.open(fpathym4)
        # convert into array
        tmpar = tmpr.read(1)
        # if integer convert into floating point
        # get transformation parameter (needed for zonal stats)
        affine = tmpr.transform
        # match year with polygon
        for filep in results_final:
            # create year filter
            if (str(int(filep.YEAR.unique()[0])) == year):
                # zonal statistics (mean)
                stats = zonal_stats(filep, tmpar, affine=affine, stats=['mean'], all_touched=True, nodata=tmpr.nodata)
                # convert dictionaries into lists
                stats1 = [val for dic in stats for val in dic.values()]
                # store in pandas dataframe
                filep[varname + '_ym4'] = stats1
        # ------- Year-5
        year_m5 = str(int(year) - 5)
        # path for raster year -1
        fpathym5 = '/ESS_Datasets/USERS/Marco/rasters/NA/dynamic/' + str.replace(varnametmp, year, year_m5) + '.tif'
        # open raster with rasterio
        tmpr = rst.open(fpathym5)
        # convert into array
        tmpar = tmpr.read(1)
        # if integer convert into floating point
        # get transformation parameter (needed for zonal stats)
        affine = tmpr.transform
        # match year with polygon
        for filep in results_final:
            # create year filter
            if (str(int(filep.YEAR.unique()[0])) == year):
                # zonal statistics (mean)
                stats = zonal_stats(filep, tmpar, affine=affine, stats=['mean'], all_touched=True, nodata=tmpr.nodata)
                # convert dictionaries into lists
                stats1 = [val for dic in stats for val in dic.values()]
                # store in pandas dataframe
                filep[varname + '_ym5'] = stats1



In [ ]:
# concatenate list of pandas dataframes
results_final1 = pd.concat(results_final)
# delete geometry column
results_final1 = results_final1.drop(['geometry'], axis=1)
# write results as csv file
results_final1.to_csv('/home/marco/Desktop/INSECT_OBSERVED.csv', index=False)